# Análise de Dados

## Market Basket Analysis


A análise de cesta de mercado é amplamente utilizada no setor de varejo, onde os dados de vendas são processados e analisados para conseguir entender os padrões de compra dos clientes e assim, alcançar as melhores estratégias de venda.

Para o presente estudo, será utilizada uma base de dados do setor varejista, com aplicação do algoritmo Apriori, que consiste na mineração de conjuntos de itens frequentes, ou seja, ele agrupa os itens que mais são comprados juntos pelos clientes, indicando suas tendências e correlações.

Dataset: 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from mlxtend.frequent_patterns import apriori, association_rules
#from mlxtend.preprocessing import TransactionEncoder
import warnings
import gc
from platform import python_version

In [2]:
warnings.filterwarnings('ignore')

In [3]:

print("Versão do python: ",python_version())

Versão do python:  3.11.5


In [4]:
tabelas = {'corredores':pd.read_csv(r"datasets\aisles.csv"), 
           'departamentos':pd.read_csv(r"datasets\departments.csv"), 
           'pedidos':pd.read_csv(r"datasets\orders.csv"), 
           'produtos':pd.read_csv(r"datasets\products.csv"), 
           'pedidos_anteriores':pd.read_csv(r"datasets\order_products__prior.csv"), 
           'pedidos_treino':pd.read_csv(r"datasets\order_products__train.csv")}

In [ ]:
# Quantidade de registros em cada tabela
contagem_registros = np.array([])
contagem_registros = pd.DataFrame([np.append(contagem_registros,[tabela[0],len(tabela[1])]) for tabela in tabelas.items()], columns = ['Tabela', 'Quantidade de registros'])
contagem_registros

In [ ]:
dados_nulos = [{tabela[0]: tabela[1].isna().sum()} for tabela in tabelas.items() if tabela[1].isna().sum().any()!=0]
dados_nulos

#### Vamos agrupar os produtos pedidos em cada ordem

In [5]:
file_path = r"datasets\order_products__prior.csv"
chunk_size = 100000
min_support = 0.01
min_confidence = 0.2
max_rows = 500000
all_frequent_itemsets = pd.DataFrame()
total_transactions = 0
count = 0

In [6]:
# Função para processar cada lote
def process_chunk(chunk):
    # Codificação dos dados
    data_encoded = pd.get_dummies(chunk.sort_values(['order_id', 'add_to_cart_order'], ascending=[True, True])[['order_id', 'product_id']].astype('int32'), columns=['product_id'])
    data_encoded.columns = data_encoded.columns.str.replace("product_id_", "")
    #display(data_encoded)
    
    # Criação do DataFrame basket
    basket = data_encoded.groupby('order_id').sum()
    basket[basket > 0] = True
    basket[basket == 0] = False

    # Aplicação do algoritmo Apriori
    frequent_itemsets = apriori(basket, min_support=min_support, use_colnames=True)
    
    return frequent_itemsets

In [7]:
for chunk in pd.read_csv(file_path, chunksize=chunk_size):
    if total_transactions >= max_rows:
        break
    
    # Limitar o número de linhas do chunk caso exceda o máximo permitido
    if total_transactions + len(chunk) > max_rows:
        chunk = chunk.iloc[:max_rows - total_transactions]
    
    # Atualizar a contagem total de transações
    total_transactions += len(chunk)
        
    count += 1
    
    # Processar o chunk e obter os frequent itemsets
    frequent_itemsets = process_chunk(chunk)
    
    # Acumular os resultados
    all_frequent_itemsets = pd.concat([all_frequent_itemsets, frequent_itemsets])
    
    # Limpar a memória
    del chunk
    gc.collect()


In [ ]:
all_frequent_itemsets_mean = all_frequent_itemsets.groupby('itemsets').agg(
    support=('support', 'mean'),
    count=('support', 'size')
).reset_index()

all_frequent_itemsets_mean['peso'] = all_frequent_itemsets_mean['count']/count
all_frequent_itemsets_mean = all_frequent_itemsets_mean[all_frequent_itemsets_mean['peso'] >= 0.6]
all_frequent_itemsets_mean = all_frequent_itemsets_mean[['itemsets', 'support']]
all_frequent_itemsets_mean

In [ ]:
rules = association_rules(all_frequent_itemsets_mean, metric="confidence", min_threshold=min_confidence)
rules = rules.sort_values(['confidence', 'lift'], ascending=[False, False])

# Exibir as regras
rules

In [ ]:
id_to_name = dict(zip(tabelas['produtos']['product_id'], tabelas['produtos']['product_name']))

In [ ]:
def map_ids_to_names(itemset):
    return frozenset([id_to_name[int(item)] for item in itemset])

In [ ]:
rules['antecedents'] = rules['antecedents'].apply(map_ids_to_names)
rules['consequents'] = rules['consequents'].apply(map_ids_to_names)

In [ ]:
rules.sort_values(by = 'confidence', ascending = False)